In [11]:
"""Example of a simple nurse scheduling problem."""
from ortools.sat.python import cp_model
import pandas as pd

In [20]:
from ortools.sat.python import cp_model
import random
import numpy as np

def main() -> None:
    # Data.
    NUM_MEMBERS = 5
    NUM_JOBS = 3
    NUM_WEEKS = 3
    all_members = range(NUM_MEMBERS)
    all_jobs = range(NUM_JOBS)
    all_weeks = range(NUM_WEEKS)

    # Simulate availability constraints (some members are unavailable on some weeks)
    availability = {
        (m, w): random.choice([True, True, False]) for m in all_members for w in all_weeks
    }
    
    # Simulate proficiency levels for each member
    proficiency = {m: random.randint(1, 10) for m in all_members}

    # Creates the model.
    model = cp_model.CpModel()

    # Creates job assignment variables.
    shifts = {}
    for m in all_members:
        for w in all_weeks:
            for j in all_jobs:
                shifts[(m, w, j)] = model.NewBoolVar(f"shift_m{m}_w{w}_j{j}")

    # Each job is assigned to exactly one member per week.
    for w in all_weeks:
        for j in all_jobs:
            model.AddExactlyOne(shifts[(m, w, j)] for m in all_members)

    # Each member does at most one job per week.
    for m in all_members:
        for w in all_weeks:
            model.AddAtMostOne(shifts[(m, w, j)] for j in all_jobs)

    # Add availability constraints
    for m in all_members:
        for w in all_weeks:
            if not availability[(m, w)]:
                for j in all_jobs:
                    model.Add(shifts[(m, w, j)] == 0)
    
    
    # Creates the solver and solve.
    solver = cp_model.CpSolver()
    solver.parameters.linearization_level = 0
    solver.parameters.enumerate_all_solutions = True

    class JobPartialSolutionPrinter(cp_model.CpSolverSolutionCallback):
        """Print intermediate solutions."""

        def __init__(self, shifts, num_members, num_weeks, num_jobs, limit):
            cp_model.CpSolverSolutionCallback.__init__(self)
            self._shifts = shifts
            self._num_members = num_members
            self._num_weeks = num_weeks
            self._num_jobs = num_jobs
            self._solution_count = 0
            self._solution_limit = limit

        def on_solution_callback(self):
            self._solution_count += 1
            print(f"Solution {self._solution_count}")
            for w in range(self._num_weeks):
                print(f"Week {w}")
                for m in range(self._num_members):
                    is_working = False
                    for j in range(self._num_jobs):
                        if self.Value(self._shifts[(m, w, j)]):
                            is_working = True
                            print(f"  Member {m} assigned to job {j}")
                    if not is_working:
                        print(f"  Member {m} has no job")
            if self._solution_count >= self._solution_limit:
                print(f"Stop search after {self._solution_limit} solutions")
                self.StopSearch()

        def solution_count(self):
            return self._solution_count

    # Display the first five solutions.
    solution_limit = 5
    solution_printer = JobPartialSolutionPrinter(
        shifts, NUM_MEMBERS, NUM_WEEKS, NUM_JOBS, solution_limit
    )

    solver.Solve(model, solution_printer)

    # Statistics.
    print("\nStatistics")
    print(f"  - conflicts      : {solver.NumConflicts()}")
    print(f"  - branches       : {solver.NumBranches()}")
    print(f"  - wall time      : {solver.WallTime()} s")
    print(f"  - solutions found: {solution_printer.solution_count()}")

if __name__ == "__main__":
    main()


Solution 1
Week 0
  Member 0 assigned to job 1
  Member 1 assigned to job 2
  Member 2 has no job
  Member 3 assigned to job 0
  Member 4 has no job
Week 1
  Member 0 assigned to job 2
  Member 1 assigned to job 0
  Member 2 assigned to job 1
  Member 3 has no job
  Member 4 has no job
Week 2
  Member 0 has no job
  Member 1 assigned to job 1
  Member 2 assigned to job 2
  Member 3 assigned to job 0
  Member 4 has no job
Solution 2
Week 0
  Member 0 assigned to job 1
  Member 1 assigned to job 2
  Member 2 has no job
  Member 3 assigned to job 0
  Member 4 has no job
Week 1
  Member 0 assigned to job 2
  Member 1 has no job
  Member 2 assigned to job 1
  Member 3 assigned to job 0
  Member 4 has no job
Week 2
  Member 0 has no job
  Member 1 assigned to job 1
  Member 2 assigned to job 2
  Member 3 assigned to job 0
  Member 4 has no job
Solution 3
Week 0
  Member 0 assigned to job 1
  Member 1 assigned to job 2
  Member 2 has no job
  Member 3 assigned to job 0
  Member 4 has no job
W